<h3> Solving A Hard SQL Problem | SQL ON OFF Problem | Magic of SQL <\h3>

In [0]:
%sql
create table event_status
(
event_time varchar(10),
status varchar(10)
);
insert into event_status 
values
('10:01','on'),('10:02','on'),('10:03','on'),('10:04','off'),('10:07','on'),('10:08','on'),('10:09','off')
,('10:11','on'),('10:12','off');

num_affected_rows,num_inserted_rows
9,9


In [0]:
%sql
select * from event_status

event_time,status
10:01,on
10:02,on
10:03,on
10:04,off
10:07,on
10:08,on
10:09,off
10:11,on
10:12,off


In [0]:
%sql
select min(event_time) as login_time ,max(event_time) logoff_time,count(1)-1 on_count  from (
select *,sum(case when status='on' and lg='off' then 1 else 0 end )over(order by event_time) as event_key from(
select *,lag(status,1)over(order by event_time) as lg
from event_status)a1
)a2
group by event_key

login_time,logoff_time,on_count
10:01,10:04,3
10:07,10:09,2
10:11,10:12,1


<h3> solution in pyspark <\h3>

In [0]:
from pyspark.sql.functions import col,max,min,lag,sum,when,count
from pyspark.sql.window import Window

In [0]:
df_e=spark.sql("select * from event_status")
df_e.show()

+----------+------+
|event_time|status|
+----------+------+
|     10:01|    on|
|     10:02|    on|
|     10:03|    on|
|     10:04|   off|
|     10:07|    on|
|     10:08|    on|
|     10:09|   off|
|     10:11|    on|
|     10:12|   off|
+----------+------+



In [0]:
df_e.withColumn("lg",lag("status",1).over(Window.orderBy("event_time"))).withColumn("event_key",\
    sum(when((col("status")=='on')&(col("lg")=='off'),1).otherwise(0)).over(Window.orderBy("event_time"))).\
        groupBy("event_key").agg(min("event_time").alias("login_time"),max("event_time").alias("logoff_time"),\
            (count("*")-1).alias("on_count")).select("login_time","logoff_time","on_count").show()

+----------+-----------+--------+
|login_time|logoff_time|on_count|
+----------+-----------+--------+
|     10:01|      10:04|       3|
|     10:07|      10:09|       2|
|     10:11|      10:12|       1|
+----------+-----------+--------+

